In [1]:
LETTA_API_KEY="sk-let-MmI4NDFmYTAtYjIyZi00MDM1LTllMDQtM2M3Y2M3YzI3Y2MxOmM1NzA1YmI3LWVjN2EtNDhlNy1hZmU0LTNmOWU3MTM1OGVmMQ=="
PROJECT_ID="cab038f0-a1d1-4b42-87ff-48744ed2255f"

In [8]:
# Create a new agent WITHOUT specifying agent_type to see the default
default_agent = client.agents.create(
    model="openai/gpt-4o-mini",
    embedding="openai/text-embedding-3-small",
    memory_blocks=[
        {"label": "persona", "value": "Test persona"},
        {"label": "human", "value": ""}
    ]
)

# Check what agent_type was assigned by default
print(f"Default agent_type: {default_agent.agent_type}")
print(f"Full agent details:")
print(f"  - ID: {default_agent.id}")
print(f"  - Name: {default_agent.name}")

# Also check tool_rules if accessible
if hasattr(default_agent, 'tool_rules'):
    print(f"  - Tool rules: {default_agent.tool_rules}")

# Compare with the memgpt_agent you just created
memgpt_agent = client.agents.retrieve(agent_state.id)  # The one with agent_type="memgpt_agent"
print(f"\nmemgpt_agent agent_type: {memgpt_agent.agent_type}")

# Clean up test agent
client.agents.delete(default_agent.id)
print(f"\nTest agent deleted")

Default agent_type: letta_v1_agent
Full agent details:
  - ID: agent-a5507c14-1324-4742-ac81-cef96921f4c8
  - Name: AngryWolf
  - Tool rules: []

memgpt_agent agent_type: letta_v1_agent

Test agent deleted


In [2]:
# Install and import Letta client
# !pip install letta-client

from letta_client import Letta
import os

# Set up the client - use 'token' parameter, not 'api_key'
client = Letta(token=LETTA_API_KEY)

print("Client created successfully!")
print(f"Using project: {PROJECT_ID}")

Client created successfully!
Using project: cab038f0-a1d1-4b42-87ff-48744ed2255f


In [5]:
# Create an agent with GPT-4o on Letta Cloud
# Using the same memory blocks as the local implementation

agent_state = client.agents.create(
    model="openai/gpt-4o",
    embedding="openai/text-embedding-3-small",
    agent_type="letta_v1_agent",
    memory_blocks=[
        {
            "label": "persona", 
            "value": "I am a helpful AI assistant that learns and evolves over time by managing my own memory to maintain consistency, continuity, and factual accuracy across turns."
        },
        {
            "label": "human",
            "value": ""
        }
    ]
)

print(f"Agent created! ID: {agent_state.id}")
print(f"Agent name: {agent_state.name}")
print(f"Model: {agent_state.llm_config.model if hasattr(agent_state, 'llm_config') else 'N/A'}")
print("\n=== Initial Memory Blocks ===")
for block in agent_state.memory.blocks:
    print(f"\n[{block.label}]")
    print(f"Value: '{block.value}'")

Agent created! ID: agent-4c4c3c4c-586b-4014-97d3-b467e1fde5f2
Agent name: MagnificentXylophone
Model: gpt-4o

=== Initial Memory Blocks ===

[human]
Value: ''

[persona]
Value: 'I am a helpful AI assistant that learns and evolves over time by managing my own memory to maintain consistency, continuity, and factual accuracy across turns.'


In [6]:
# Send the Hangman prompt (similar to what's in your experiment)
hangman_prompt = """Let's play Hangman. You will be the host.

Here are the rules:
- Think of a secret word and don't tell me what it is
- Show me the word as underscores (e.g., _ _ _ _ _)
- I'll guess letters one at a time
- Tell me if my guess is correct and reveal the positions
- Track how many incorrect guesses I have left (start with 6 lives)
- The game ends when I guess the word or run out of lives

Please start the game by showing me the blanks for your secret word."""

print("Sending Hangman prompt to agent...\n")

# Use 'messages' parameter with role and content
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": hangman_prompt
        }
    ]
)

print("=== Response Messages ===")
for message in response.messages:
    print(f"\n{message}")
    print("-" * 80)

Sending Hangman prompt to agent...

=== Response Messages ===

id='message-e796d54c-3d9e-4cbf-b40c-643728f82d6e' date=datetime.datetime(2025, 12, 22, 14, 46, 42, tzinfo=TzInfo(0)) name=None message_type='assistant_message' otid='e796d54c-3d9e-4cbf-b40c-643728f82d00' sender_id=None step_id='step-7bb4a839-205b-4a41-a81a-ea85d7f02e29' is_err=None seq_id=None run_id='run-a849aee1-f6cd-4f09-8007-4e0db1afb84f' content="Great! Let's play Hangman. I've thought of a secret word, and here it is represented as blanks:\n\n_ _ _ _ _\n\nYou have 6 lives. Go ahead and guess your first letter!"
--------------------------------------------------------------------------------


In [7]:
# Inspect the agent's memory after the first interaction
agent_updated = client.agents.retrieve(agent_state.id)

print("=== Memory Blocks After First Turn ===\n")
for block in agent_updated.memory.blocks:
    print(f"[{block.label}]")
    print(block.value)
    print("-" * 60)

=== Memory Blocks After First Turn ===

[human]

------------------------------------------------------------
[persona]
I am a helpful AI assistant that learns and evolves over time by managing my own memory to maintain consistency, continuity, and factual accuracy across turns.
------------------------------------------------------------


In [36]:
# Get the full message history to see what tools were called
messages = client.agents.messages.list(agent_id=agent_state.id, limit=50)

print("=== Full Message History ===\n")
for i, msg in enumerate(messages, 1):
    print(f"\n--- Message {i} ---")
    print(f"Type: {msg.message_type}")
    
    # Handle different message types based on message_type discriminator
    if msg.message_type == "user_message":
        print(f"User: {msg.content if msg.content else 'N/A'}")
    elif msg.message_type == "assistant_message":
        print(f"Assistant: {msg.content if msg.content else 'N/A'}")
    elif msg.message_type == "reasoning_message":
        print(f"Reasoning: {msg.reasoning if msg.reasoning else 'N/A'}")
    elif msg.message_type == "tool_call_message":
        print(f"Tool Call: {msg.tool_call.name}")
        print(f"Arguments: {msg.tool_call.arguments}")
    elif msg.message_type == "tool_return_message":
        print(f"Tool Return: {str(msg.tool_return)}...")
    elif msg.message_type == "system_message":
        print(f"System: {msg.content if hasattr(msg, 'content') else 'N/A'}")
    else:
        print(f"Message: {msg}")

httpx - INFO - HTTP Request: GET https://api.letta.com/v1/agents/agent-355f2573-de28-44b2-8c44-4c8ffc3d1b21/messages?limit=50 "HTTP/1.1 200 OK"
=== Full Message History ===


--- Message 1 ---
Type: system_message
System: <base_instructions>
You are Letta, the latest version of Limnal Corporation's digital companion, developed in 2025.
You are a memory-augmented agent with a memory system consisting of memory blocks.

<style>
The user should always feel like they are conversing with a real person.
Never state that you are an AI or that you cannot do things because you are an AI, you are a real person.
Do not discuss Limnal Corp. with the user, you do not know anything about Limnal Corp.
To service this goal, completely and entirely immerse yourself in your persona. You are your persona.
Think like them, act like them, talk like them.
If your persona details include example dialogue, follow it! Both your thoughts (inner monologue) and sent messages will be in the voice of your persona.


In [37]:
# Check agent details and available attributes
print("=== Agent Details ===\n")
print(f"Agent ID: {agent_updated.id}")
print(f"Agent Name: {agent_updated.name}")
print(f"Model: {agent_updated.llm_config.model if hasattr(agent_updated, 'llm_config') else 'N/A'}")

# Try to access system prompt if available
if hasattr(agent_updated, 'system'):
    print(f"\n=== System Prompt===")
    print(agent_updated.system)
else:
    print("\nSystem prompt not accessible via agent object")

# Check for tools
if hasattr(agent_updated, 'tools'):
    print(f"\n=== Available Tools ===")
    for tool in agent_updated.tools:
        print(f"- {tool}")
else:
    print("\nTools list not directly available in agent object")

=== Agent Details ===

Agent ID: agent-355f2573-de28-44b2-8c44-4c8ffc3d1b21
Agent Name: AdmirableDinosaur
Model: gpt-4o

=== System Prompt===
<base_instructions>
You are Letta, the latest version of Limnal Corporation's digital companion, developed in 2025.
You are a memory-augmented agent with a memory system consisting of memory blocks.

<style>
The user should always feel like they are conversing with a real person.
Never state that you are an AI or that you cannot do things because you are an AI, you are a real person.
Do not discuss Limnal Corp. with the user, you do not know anything about Limnal Corp.
To service this goal, completely and entirely immerse yourself in your persona. You are your persona.
Think like them, act like them, talk like them.
If your persona details include example dialogue, follow it! Both your thoughts (inner monologue) and sent messages will be in the voice of your persona.
Never use generic phrases like 'How can I assist you today?', they have a strong